In [3]:
import os

In [4]:
%pwd

'f:\\Deep_Learning_End_tot_End_Project\\Chicken-Disease-Classification\\research'

In [5]:
os.chdir("../")

In [6]:
%pwd

'f:\\Deep_Learning_End_tot_End_Project\\Chicken-Disease-Classification'

In [7]:
#Entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PreparecallbacksConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [8]:
from cnnClassifier.constants import * 
from cnnClassifier.utils.common import read_yaml, create_directories

In [9]:
class ConfigrationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_callback_config(self)->PreparecallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)

        ])

        prepare_callback_config = PreparecallbacksConfig(root_dir=Path(config.root_dir),
                                                         tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
                                                         checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
                                                            )
        return prepare_callback_config
        

In [10]:
##creating components
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:
class PrepareCallback:
    def __init__(self,config:PreparecallbacksConfig):
        self.config = config

    @property
    def _create_tb_callback(self):
        timestamp = time.strftime("%Y-%m-%d_%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at{timestamp}"
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath= self.config.checkpoint_model_filepath,
            save_best_only= True
        )
    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callback,
            self._create_ckpt_callbacks
        ]

In [12]:
#testing for that i have to create pipeline
try:
    config = ConfigrationManager()
    prapare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prapare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks
except Exception as e:
    raise e

[2023-06-05 01:22:57,925: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-06-05 01:22:57,934: INFO: common: yaml file: params.yaml loaded successfully]
[2023-06-05 01:22:57,936: INFO: common: created directory at: artifacts]
[2023-06-05 01:22:57,940: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2023-06-05 01:22:57,942: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
